# Get Data for D3 Graphing

-------------------------------------------------

The raw code for this Jupyter notebook is by default hidden for easier reading. The main focus of this particular page of the notebook is on the graphs and their interpretation. To toggle on/off the raw code, click below:

In [1]:
# Setup Code toggle button
from IPython.core.display import HTML  

HTML(''' 
<center><h3>
<a href="javascript:code_toggle()">Talk is cheap, show me the code.</a>
</center></h3>
<script>
    var code_show=true; //true -> hide code at first

    function code_toggle() {
        $('div.prompt').hide(); // always hide prompt

        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
        }
        code_show = !code_show
    }
    $( document ).ready(code_toggle);
</script>
''')

In [2]:
# Setup notebook theme
from jupyterthemes import get_themes
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme(get_themes()[1])

&nbsp;

## Get the Data

For a list of `python-twitter` `User` class attributes see: [https://python-twitter.readthedocs.io/en/latest/_modules/twitter/models.html#User.NewFromJsonDict](https://python-twitter.readthedocs.io/en/latest/_modules/twitter/models.html#User.NewFromJsonDict).

First fetch friends using [GET followers](https://dev.twitter.com/rest/reference/get/followers/list).

&nbsp;

In [3]:
from followers import Followers

uid = 151901549
f = Followers(uid)
users = f.api.GetFollowers(uid)

users

[User(ID=59860290, ScreenName=ryacca),
 User(ID=9421452, ScreenName=shebin),
 User(ID=265517884, ScreenName=ycpdan),
 User(ID=3979114762, ScreenName=BexAnnalisa),
 User(ID=756135755190198273, ScreenName=degruyter_lib),
 User(ID=3158073120, ScreenName=mixnode),
 User(ID=481260788, ScreenName=no_identd),
 User(ID=1429810189, ScreenName=JCrueger),
 User(ID=4008474733, ScreenName=lissertations),
 User(ID=821042028800802816, ScreenName=Grant_ODU),
 User(ID=827268804359053313, ScreenName=mmicros1),
 User(ID=789293970677456896, ScreenName=74paddycakes),
 User(ID=799272100599046145, ScreenName=digitopia_nl),
 User(ID=203096228, ScreenName=PruittAL),
 User(ID=81650379, ScreenName=brendaberkelaar),
 User(ID=47899646, ScreenName=CassPF),
 User(ID=899863316, ScreenName=OKMotovlog),
 User(ID=30802131, ScreenName=mekonkol),
 User(ID=761396027446398976, ScreenName=Stesker3D),
 User(ID=825218681252765696, ScreenName=HussamHallak1),
 User(ID=20255236, ScreenName=carolc16),
 User(ID=825049390998024192, 

&nbsp;

We want the subject in the graph too so append the `User` object, then create a dictionary to hold the nodes and links between them. From each `User` object put only the needed data into a `dict` and append it to the list of nodes.

&nbsp;

In [4]:
users.append(f.api.GetUser(user_id=uid))

data = { 'nodes' : [], 'links' : [] }
for user in users:
    user_data = {   'id': user.id,
                    'name': user.name,
                    'profile_image_url': user.profile_image_url,
                    'screen_name': user.screen_name,
                    'default_profile_image': user.default_profile_image
                }
    data['nodes'].append(dict(user_data))
    
data['nodes']

[{'default_profile_image': False,
  'id': 59860290,
  'name': 'Ryan Yacyshyn',
  'profile_image_url': 'http://pbs.twimg.com/profile_images/796184339687931905/--nSmCcF_normal.jpg',
  'screen_name': 'ryacca'},
 {'default_profile_image': False,
  'id': 9421452,
  'name': 'Shebin',
  'profile_image_url': 'http://pbs.twimg.com/profile_images/608858046731673600/nwm8reMF_normal.jpg',
  'screen_name': 'shebin'},
 {'default_profile_image': False,
  'id': 265517884,
  'name': 'Daniel',
  'profile_image_url': 'http://pbs.twimg.com/profile_images/2839957630/6742066915c916b9448b513314bab9f2_normal.jpeg',
  'screen_name': 'ycpdan'},
 {'default_profile_image': False,
  'id': 3979114762,
  'name': 'Rebecca Dallavalle',
  'profile_image_url': 'http://pbs.twimg.com/profile_images/822219089049452546/yjbE3trx_normal.jpg',
  'screen_name': 'BexAnnalisa'},
 {'default_profile_image': False,
  'id': 756135755190198273,
  'name': 'De Gruyter Libraries',
  'profile_image_url': 'http://pbs.twimg.com/profile_imag

&nbsp;

Now to determine to determine the the links between each `User` we can use [https://dev.twitter.com/rest/reference/get/friendships/show](https://dev.twitter.com/rest/reference/get/friendships/show).

&nbsp;

In [5]:
f.api.ShowFriendship(source_user_id=uid, target_screen_name=users[5].screen_name)

{'relationship': {'source': {'all_replies': None,
   'blocked_by': None,
   'blocking': None,
   'can_dm': True,
   'followed_by': True,
   'following': False,
   'following_received': None,
   'following_requested': None,
   'id': 151901549,
   'id_str': '151901549',
   'live_following': False,
   'marked_spam': None,
   'muting': None,
   'notifications_enabled': None,
   'screen_name': 'phonedude_mln',
   'want_retweets': None},
  'target': {'followed_by': False,
   'following': True,
   'following_received': None,
   'following_requested': None,
   'id': 3158073120,
   'id_str': '3158073120',
   'screen_name': 'mixnode'}}}

&nbsp;

Wrap this up in a function for use in a loop to get links.

&nbsp;

In [ ]:
import json

def friends(src_username, target_username):
    friendship = f.api.ShowFriendship(source_screen_name=src_username, \
                                      target_screen_name=target_username)

    following = friendship['relationship']['source']['following']
    followed = friendship['relationship']['source']['followed_by']        
    return (following, followed)

friends('phonedude_mln', users[10].screen_name)

(False, True)

&nbsp;

Iterate over all users to generate the link data. Note that the `python-twitter` API is instantiated with `sleep_on_rate_limit=True` so that will be the cause if this takes a long time. Use of `itertools.combinations()` ensures no wasted calls eat up more time than needed.

&nbsp;

In [ ]:
from time import sleep
from itertools import combinations

for alice, bob in combinations(users, 2):
    friendship = friends(alice.screen_name, bob.screen_name) #?
    if friendship[0]: # alice following bob
        link = { 'source': alice.screen_name,
                 'target': bob.screen_name
               }
        data['links'].append(dict(link))
    if friendship[1]: # alice followed by bob
        link = { 'source': bob.screen_name,
                 'target': alice.screen_name
               }
        data['links'].append(dict(link))
    sleep(5.05) # for bug in python-twitter rate-limit timing (23 Mär 2017)

print(data)

# This data is precious time write it out now
data_file = 'graph_data.json'
with open(data_file, 'w') as outfile:
    json.dump(data, outfile)

In [ ]:
with open(data_file, 'r') as infile:
    complete_data = json.load(infile)